# 1. Xây dựng mạng nơ ron trên tensorflow

Trong bài [giới thiệu tensorflow](https://www.kaggle.com/phamdinhkhanh/tensorflow-turtorial?scriptVersionId=6702388) chúng ta đã học được cách tạo ra một mạng nơ ron theo phương pháp code bậc thấp. Quá trình xây dựng các layer chúng ta phải xây dựng những bước cơ bản nhất như khởi tạo hệ số, phép nhân ma trận chuyển hóa layer, tính toán xác xuất đầu ra, xây dựng hàm loss function,.... Điều đó gây ra khó khăn cho các data scientist rất nhiều vì quá trình khởi tạo một mạng nơ ron nông là khả thi nhưng những mạng nơ ron sâu với vài trăm ngàn tầng ẩn việc code chay mỗi layer là bất khả thi. Chính vì thế chúng ta cần sự hỗ trợ của các thư viện bậc cao để khởi tạo các layer và huấn luyện, đánh giá và dự báo mô hình một cách dễ dàng hơn. Thông qua bài viết này chúng ta sẽ biết cách khởi tạo một mạng nơ ron như thế thông qua một class rất toàn năng. Đó chính là Estimator.
Bên dưới là các bước xây dựng mạng nơ ron.

# 2. Hàm xử lý dữ liệu đầu vào.

Ở hàm này ta sẽ trả về một Dataset hoặc một Iterator Dataset của tensorflow từ đường link và tên file dữ liệu của tập train và test. 

* **Dataset** sẽ có tác dụng quản lý quá trình input dữ liệu đầu vào gồm các thành phần là một hoặc nhiều Tensors chẳng hạn như: Kích thước batch_size, số lần shuffle mẫu, lấy mẫu tái lặp hay không?.... 
* **Iterator Dataset** cũng gần giống như enumerate cho phép chúng ta truy cập vào các thành phần trong Dataset để thực hiện các vòng lặp khi huấn luyện qua các batches khác nhau đánh giá mô hình qua các tập dữ liệu khác nhau.

Trong trường hợp tập dữ liệu là tập train chúng ta sẽ sử dụng Iterator Dataset do quá trình huấn luyện và đánh giá thuật toán gradient descent cần thực hiện trên nhiều batches khác nhau. Tập dữ liệu test là một tập dữ liệu cố định sử dụng cho dự báo nên không cần phải là một Iterator Dataset.

In [1]:
import os
import io
import urllib.request as request
import numpy as np
import pandas as pd
import tensorflow as tf

#Khai báo params
IRIS_TRAIN = 'iris_training.csv'
IRIS_TRAIN_PATH = 'https://raw.githubusercontent.com/phamdinhkhanh/Tensorflow/master/iris_training.csv'
IRIS_TEST = 'iris_testing.csv'
IRIS_TEST_PATH  = 'https://raw.githubusercontent.com/phamdinhkhanh/Tensorflow/master/iris_testing.csv'
COLUMNS = ['SepWid', 'SepLen', 'PenWid', 'PenLen', 'Species']
BATCH_SIZE = 100
N_STEPS = 1000
LEARNING_RATE = 0.2

def get_data(url, filename, is_get_train = True):
    if not os.path.exists(filename):
        raw = request.urlopen(url).read().decode('utf-8')
        with io.open(filename, 'w') as f:
            f.write(raw)
            
    data = pd.read_csv(filename, header = 0, names = COLUMNS, encoding = 'utf-8')
    features, labels = data, data.pop('Species')
    
    #Tạo Class Dataset trong tensorflow
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if is_get_train: 
        dataset = dataset.shuffle(1000).repeat().batch(batch_size = BATCH_SIZE)
        return dataset.make_one_shot_iterator().get_next()
    else:
        return dataset.batch(batch_size = BATCH_SIZE)

get_data(IRIS_TRAIN_PATH, IRIS_TRAIN, is_get_train = True)
get_data(IRIS_TEST_PATH, IRIS_TEST,  is_get_train = False)

<BatchDataset shapes: ({SepWid: (?,), SepLen: (?,), PenWid: (?,), PenLen: (?,)}, (?,)), types: ({SepWid: tf.float64, SepLen: tf.float64, PenWid: tf.float64, PenLen: tf.float64}, tf.int64)>

Đọc dữ liệu từ Dataset của tensorflow thông qua một session.

In [2]:
with tf.Session() as sess:
    test = get_data(IRIS_TEST_PATH, IRIS_TEST, is_get_train = False)
    print(sess.run(test.make_one_shot_iterator().get_next()))

({'SepWid': array([5.9, 6.9, 5.1, 6. , 5.5, 6.2, 5.5, 6.3, 5.6, 6.7, 7.1, 4.3, 5.6,
       5.5, 6. , 5.1, 5.7, 4.8, 5.1, 5.7, 5.4, 5.6, 6.3, 6.3, 5.8, 6.1,
       5.2, 6.7, 6.7, 6.4]), 'SepLen': array([3. , 3.1, 3.3, 3.4, 2.5, 2.9, 4.2, 2.8, 3. , 2.5, 3. , 3. , 2.8,
       2.3, 2.2, 3.5, 2.6, 3.4, 3.4, 2.5, 3.4, 3. , 2.9, 2.5, 2.7, 3. ,
       4.1, 3.1, 3.3, 2.9]), 'PenWid': array([4.2, 5.4, 1.7, 4.5, 4. , 4.3, 1.4, 5.1, 4.1, 5.8, 5.9, 1.1, 4.9,
       4. , 4. , 1.4, 3.5, 1.9, 1.5, 5. , 1.7, 4.5, 5.6, 4.9, 3.9, 4.6,
       1.5, 4.7, 5.7, 4.3]), 'PenLen': array([1.5, 2.1, 0.5, 1.6, 1.3, 1.3, 0.2, 1.5, 1.3, 1.8, 2.1, 0.1, 2. ,
       1.3, 1. , 0.2, 1. , 0.2, 0.2, 2. , 0.2, 1.5, 1.8, 1.5, 1.2, 1.4,
       0.1, 1.5, 2.5, 1.3])}, array([1, 2, 0, 1, 1, 1, 0, 2, 1, 2, 2, 0, 2, 1, 1, 0, 1, 0, 0, 2, 0, 1,
       2, 1, 1, 1, 0, 1, 2, 1], dtype=int64))


# 3. Xây dựng mạng nơ ron.

Ta sẽ xây dựng 1 hàm `my_model` có 3 chức năng gồm: `huấn luyện mô hình, đánh giá mô hình và dự báo mô hình`. Class được sử dụng để khởi tạo mô hình này là `tf.estimator.Estimator`.

In [3]:
def my_model(features, labels, mode, params):
   
    # 0. Xây dựng mạng nơ ron
    # Khởi tạo input_layer. Hàm input_layer sẽ map dữ liệu đầu vào là features với các estimators thông qua params['features_columns']
    nn = tf.feature_column.input_layer(features, params['feature_columns'])
    # Xây dựng các hidden layer tiếp theo
    for n_units in params['hidden_units']:
        nn = tf.layers.dense(nn, n_units, activation = tf.nn.relu)
    # Hàm logits dự báo xác xuất các classes (chính là output layer)
    logits = tf.layers.dense(nn, params['n_classes'], activation = tf.nn.softmax)
    if labels is not None:
        # Hàm loss function
        loss = tf.losses.sparse_softmax_cross_entropy(
            labels = labels, 
            logits = logits)
    
    #---------------------------------------------------------------------------------
    # 1. Huấn luyện mô hình
    if mode == tf.estimator.ModeKeys.TRAIN:
        # Hàm tối ưu hóa kiểm soát thuật toán gradient descent:
        optimizer = tf.train.AdagradOptimizer(learning_rate = LEARNING_RATE)
        # Hàm kích hoạt quá trình training mô hình:
        train_op = optimizer.minimize(
                   loss, 
                   global_step = tf.train.get_global_step())
        # Estimator trả về 
        return tf.estimator.EstimatorSpec(
            mode = mode, 
            loss = loss, 
            train_op = train_op)
    
    #---------------------------------------------------------------------------------
    # 2. Đánh giá mô hình
    elif mode == tf.estimator.ModeKeys.EVAL:
        # Lớp được dự báo 
        prediction_classes = tf.argmax(logits, 1)
        # Mức độ chính xác 
        accuracy = tf.metrics.accuracy(
            labels = labels,
            predictions = prediction_classes
        )
        # Estimator trả về
        return tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {'accuracy': accuracy}
        )
    #----------------------------------------------------------------------------------
    # 3. Dự báo mô hình
    if mode == tf.estimator.ModeKeys.PREDICT:
        #Tính class dự báo.
        predicted_class = tf.argmax(logits, 1)
        # Estimator trả về
        return tf.estimator.EstimatorSpec(
            mode = mode, 
            predictions = {
                'class_id': predicted_class,
                'logits': logits,
                'probabilities': tf.nn.softmax(logits)
            })

Chúng ta tưởng tượng Dataset của tensorflow như một kho chứa nguyên liệu trong đó gồm 2 nguyên liệu chính là các features và labels. Các estimator trong mạng nơ ron như một đường ống dẫn nguyên liệu. Để nối đúng loại nguyên liệu với đúng đường ống chúng ta cần một danh sách các features để map giữa Dataset và các estimator trong mạng nơ ron trong input_layer. 

In [4]:
my_features = []
for column in COLUMNS[:-1]:
    my_features.append(tf.feature_column.numeric_column(column))
my_features

[_NumericColumn(key='SepWid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenWid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Hàm tf.feature_column.numeric_column() sẽ tạo ra các phần tử định dạng numeric trong list danh sách features. Xây dựng object classifier thuộc class Estimator sử dụng để huấn luyện, đánh giá và dự báo.

In [5]:
classifier = tf.estimator.Estimator(
    model_fn = my_model,
    params = {
        'feature_columns': my_features, # List tên các feature sử dụng để map dữ liệu từ Dataset với các estimator
        'hidden_units':[10, 20, 10], # Số đơn vị mỗi layer
        'n_classes': 3 # Số lượng nhóm cần phân loại
    }
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LAPTOP~1\\AppData\\Local\\Temp\\tmp5jncfld7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018ACF42B7F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Ta nhận thấy classifier sử dụng model function là my_model để xây dựng mạng nơ ron, huấn luyện, đánh giá và dự báo mô hình. Tham số params của hàm Estimator sẽ được truyền vào trong hàm my_model để khai thác các thông số bao gồm: 
* feature_columns: Danh sách các biến dự báo được sử dụng để map dữ liệu từ Dataset với features đầu vào của mô hình.
* hidden_units: Số lượng units mỗi tầng ẩn. Mạng nơ ron của chúng ta sẽ được thiết kế gồm 3 tầng ẩn với số units lần lượt là 10, 20, 10.
* n_classes: Số nhóm phân loại. 
Tiếp theo chúng ta sẽ huấn luyện mô hình

# 4. Huấn luyện mô hình.

Hàm huấn luyện sẽ cần truyền vào 2 tham số chính đó là hàm khởi tạo Dataset để truyền dữ liệu và số bước học tập.

In [6]:
classifier.train(
    input_fn = lambda:get_data(IRIS_TRAIN_PATH, IRIS_TRAIN),
    steps = N_STEPS
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt.
INFO:tensorflow:loss = 1.1937896, step = 1
INFO:tensorflow:global_step/sec: 188.223
INFO:tensorflow:loss = 0.656974, step = 101 (0.535 sec)
INFO:tensorflow:global_step/sec: 275.244
INFO:tensorflow:loss = 0.5916569, step = 201 (0.365 sec)
INFO:tensorflow:global_step/sec: 267.529
INFO:tensorflow:loss = 0.5698997, step = 301 (0.372 sec)
INFO:tensorflow:global_step/sec: 290.742
INFO:tensorflow:loss = 0.57975054, step = 401 (0.344 sec)
INFO:tensorflow:global_step/sec: 297.355
INFO:tensorflow:loss = 0.60341954, step = 501 (0.336 sec)
INFO:tensorflow:global_step/sec: 264.559
INFO:tensorflow:loss = 0.57738477, step = 601 (0.378 sec)
INFO:tensorflow:global_

# 5. Đánh giá mô hình.
Ta sẽ đánh giá mô hình lần lượt trên tập train và tập test. Hàm đánh giá mô hình sẽ trả về mức độ dự báo chính xác của mạng nơ ron trên từng tập dữ liệu. Tham số đầu vào input_fn là một hàm trả về một Dataset hoặc Iterator Dataset của features và labels.

In [7]:
classifier.evaluate(
    input_fn = lambda:get_data(IRIS_TEST_PATH, IRIS_TEST, is_get_train = False)
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-25-13:32:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-25-13:32:48
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, global_step = 1000, loss = 0.58936626
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt-1000


{'accuracy': 0.96666664, 'global_step': 1000, 'loss': 0.58936626}

Đánh giá mô hình trên tập train

In [8]:
classifier.evaluate(
    input_fn = lambda:get_data(IRIS_TRAIN_PATH, IRIS_TRAIN, is_get_train = False)
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-25-13:32:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-25-13:32:49
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.98333335, global_step = 1000, loss = 0.56490624
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt-1000


{'accuracy': 0.98333335, 'global_step': 1000, 'loss': 0.56490624}

Với một mạng nơ ron chỉ 3 tầng ẩn kết quả chính xác đã > 95%. Đây là một kết quả khá tốt mà những mô hình học máy khác như `Hồi qui Logistic, kNN, SVM, Random forest, Decesion Tree,...` khó đạt được. Nếu bạn đọc không tin điều này có thể search thông tin kết quả các thuật toán học máy áp dụng trên bộ dữ liệu iris để kiểm chứng.

# 6. Dự báo quan sát mới.
Giả sử bên dưới chúng ta có 3 quan sát về kích thước dài, rộng của cánh hoa và đài hoa. Chúng ta sẽ sử dụng hàm predict của object classifier để dự báo nhãn và xác xuất của 3 quan sát mới này. Lưu ý hàm predict, train, evaluate đều có chung một đặc điểm đó là có một tham số là hàm số trả về Dataset của tensorflow hoặc Iterator Dataset.

In [9]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepLen': [3.1, 5.9, 6.9],
    'SepWid': [2.3, 3.0, 3.1],
    'PenLen': [1.7, 2.2, 5.4],
    'PenWid': [0.5, 1.5, 2.1],
}

def input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

def pred(input_features):
    predictions = classifier.predict(
        input_fn=lambda:input_fn(input_features, labels = None, batch_size = BATCH_SIZE))
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
    for pred in list(predictions):
        class_id = pred['class_id']
        exp = expected[class_id]
        prob = pred['probabilities'][class_id]*100
        print(template.format(class_id, prob, exp))

pred(predict_x)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LAPTOP~1\AppData\Local\Temp\tmp5jncfld7\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "1" (49.7%), expected "Versicolor"

Prediction is "1" (57.6%), expected "Versicolor"

Prediction is "2" (57.6%), expected "Virginica"


# 7. Tài liệu tham khảo.

1. [Estimator - tensorflow](https://www.tensorflow.org/api_docs/python/tf/estimator)
2. [Dataset - tensorflow](https://www.tensorflow.org/guide/datasets)
3. [Layer - tensorflow](https://www.tensorflow.org/api_docs/python/tf/layers)
4. [Hướng dẫn tensorflow - Pham Dinh Khanh](https://www.kaggle.com/phamdinhkhanh/tensorflow-turtorial?scriptVersionId=6702388)
5. [Quá trình lan truyền ngược và lan truyền thuận - Blog Machine learning cơ bản](https://machinelearningcoban.com/2017/02/24/mlp/)
6. [Các thuật toán Gradient Descent - tensorflow](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer)
7. [Loạt bài về tensorflow - Nguyễn Văn Hiếu](https://nguyenvanhieu.vn/xay-dung-mo-hinh-neural-network/)